In [1]:
# Mount Google Drive in the Colab environment
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
reviews_data = []
labels_data = []

In [4]:
files = os.listdir('/content/drive/MyDrive/RNN/')
for file in files:
    if file == 'reviews.txt':
        file = open(os.path.join('/content/drive/MyDrive/RNN', file), 'r')
        while True:
            next_line = file.readline()
            reviews_data.append(next_line.replace('\n', '').strip().split())
            if not next_line:
                break;
        file.close()
    elif file == 'labels.txt':
        file = open(os.path.join('/content/drive/MyDrive/RNN', file), 'r')
        while True:
            next_line = file.readline()
            labels_data.append([0] if next_line.replace('\n', '').strip() == 'positive' else [1])
            if not next_line:
                break;
        file.close()

In [5]:
training_size = 20000

training_sentences = reviews_data[0:training_size]
testing_sentences = reviews_data[training_size:]
training_labels = labels_data[0:training_size]
testing_labels = labels_data[training_size:]

In [6]:
vocab_size = 10000
max_length = 512
padding_type = 'post'
trunc_type = 'post'

tokenizer = Tokenizer(num_words=vocab_size, oov_token = "<OOV>")
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [7]:
training_padded_numpy = np.array(training_padded)
training_labels_numpy = np.array(training_labels)
testing_padded_numpy = np.array(testing_padded)
testing_labels_numpy = np.array(testing_labels)

In [8]:
word_index

{'<OOV>': 1,
 'the': 2,
 '.': 3,
 'and': 4,
 'a': 5,
 'of': 6,
 'to': 7,
 'is': 8,
 'br': 9,
 'it': 10,
 'in': 11,
 'i': 12,
 'this': 13,
 'that': 14,
 's': 15,
 'was': 16,
 'as': 17,
 'for': 18,
 'with': 19,
 'movie': 20,
 'but': 21,
 'film': 22,
 'on': 23,
 'you': 24,
 't': 25,
 'not': 26,
 'he': 27,
 'are': 28,
 'his': 29,
 'have': 30,
 'be': 31,
 'one': 32,
 'all': 33,
 'at': 34,
 'they': 35,
 'by': 36,
 'an': 37,
 'who': 38,
 'so': 39,
 'from': 40,
 'like': 41,
 'her': 42,
 'there': 43,
 'or': 44,
 'just': 45,
 'about': 46,
 'out': 47,
 'if': 48,
 'has': 49,
 'what': 50,
 'some': 51,
 'good': 52,
 'she': 53,
 'can': 54,
 'more': 55,
 'when': 56,
 'very': 57,
 'up': 58,
 'time': 59,
 'no': 60,
 'would': 61,
 'my': 62,
 'even': 63,
 'which': 64,
 'story': 65,
 'only': 66,
 'really': 67,
 'see': 68,
 'had': 69,
 'their': 70,
 'were': 71,
 'we': 72,
 'me': 73,
 'well': 74,
 'much': 75,
 'than': 76,
 'been': 77,
 'get': 78,
 'people': 79,
 'will': 80,
 'other': 81,
 'do': 82,
 'great':

In [9]:
multiLayerModel = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [10]:
multiLayerModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          640000    
                                                                 
 bidirectional (Bidirection  (None, None, 128)         66048     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 751489 (2.87 MB)
Trainable params: 751489 

In [11]:
multiLayerModel.compile(loss='binary_crossentropy',
                        optimizer='adam',
                        metrics=['accuracy'])

In [ ]:
history = multiLayerModel.fit(
            training_padded_numpy,
            training_labels_numpy,
            validation_data=(testing_padded_numpy, testing_labels_numpy),
            epochs=20,
            verbose=1)

Epoch 1/20
625/625 [==============================] - 610s 964ms/step - loss: 0.5784 - accuracy: 0.6911 - val_loss: 0.5442 - val_accuracy: 0.7475
Epoch 2/20
429/625 [===================>..........] - ETA: 2:48 - loss: 0.3820 - accuracy: 0.8413

In [ ]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
sentences = ['Simply awful and disappointed! It’s a very shallow story, and there in no time to feel an affinity for the main character/characters. It’s almost like whoever edited this movie, edited the first 30% and the last 30% and the movie begins as if you missed part of it. It ends, as if the signal on your TV went out, and it’s abrupt snd makes no sense! OMG! Wait for this movie to come out on DVD! HONESTLY! I am an avid movie goer!',
             'Great movie overall! Hoping for a 2nd one to pickup where the ending left off. Left with a cliffhanger so fingers crossed.']

sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
predictions = multiLayerModel.predict(padded)

for i in range(len(predictions)):
    if predictions[i] <= 0.5:
        print('POSITIVE')
    else:
        print('NEGATIVE')